In [1]:
import ee

# Initialize Earth Engine
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [2]:
import random

# Initialize the scheduler and burned sets
scheduler = {
    0: {(0, 0)}
}

# Controlled burn
burned = set([(i, 2) for i in range(-30, 30)]+[(i, -2) for i in range(-30, 30)]+[(2, i) for i in range(-30, 30)]+[(-2, i) for i in range(-30, 30)])

# Task tracking
last_task = 0
cur_task = 0

def calc_prob(node1, node2):
    return .5

def calc_time(node1, node2):
    return 1

# Main simulation loop
while cur_task <= last_task:
    if cur_task in scheduler:
        for node in scheduler[cur_task]:
            for lon_dist in range(-1, 2):
                for lat_dist in range(-1, 2):
                    if lon_dist == 0 and lat_dist == 0:
                        continue
                    next_node = (node[0] - lon_dist, node[1] - lat_dist)
                    if next_node in burned:
                        continue
                    if random.random() > calc_prob(node, next_node):
                        continue
                    next_time = calc_time(node, next_node) + cur_task
                    last_task = max(last_task, next_time)
                    if next_time in scheduler:
                        scheduler[next_time].add(next_node)
                    else:
                        scheduler[next_time] = {next_node}
            burned.add(node)
    cur_task += 1

# Output the results
print(scheduler)
print(burned)


{0: {(0, 0)}, 1: {(0, 1), (-1, -1), (-1, 1), (1, 1), (0, -1)}, 2: {(1, 1), (1, -1), (-1, 0), (1, 0), (0, -1)}, 3: {(1, 0)}}
{(-11, 2), (-30, -2), (-19, -2), (-2, 14), (2, -25), (-2, 23), (2, -16), (2, -7), (-9, 2), (2, 2), (-17, -2), (2, 11), (-2, -11), (6, 2), (-13, -2), (-2, -2), (-1, -2), (-2, -1), (-1, -1), (-2, 7), (4, 2), (-2, 16), (2, -23), (-2, 25), (2, -14), (8, 2), (19, 2), (0, -2), (11, -2), (2, -5), (0, -1), (2, 4), (-2, -18), (15, -2), (26, -2), (-2, -9), (21, 2), (-2, 0), (-1, 0), (-2, 9), (2, 25), (-19, 2), (-30, 2), (2, -30), (-27, -2), (-2, 18), (17, -2), (28, -2), (2, -21), (2, -12), (-23, -2), (2, -3), (0, 0), (-2, -25), (-17, 2), (-2, -16), (-2, -7), (-2, 2), (-1, 2), (-13, 2), (-21, -2), (-10, -2), (-2, 11), (2, 27), (2, -28), (-6, -2), (5, -2), (2, -19), (2, -10), (0, 2), (11, 2), (-8, -2), (-2, -23), (-2, -14), (15, 2), (26, 2), (-4, -2), (7, -2), (18, -2), (-2, -5), (-2, 4), (2, 20), (22, -2), (2, 29), (2, -26), (-27, 2), (17, 2), (28, 2), (2, -17), (9, -2), (2,

In [3]:
print(burned)
import geemap
Map = geemap.Map()

wildfire_lat = 39.819
wildfire_lon = -121.419
node_dist = .0009

# Inside your loop, after calculating the next burning cells
# Convert your burned cells to a FeatureCollection
burned_feature_collection = ee.FeatureCollection([ee.Feature(ee.Geometry.Point(wildfire_lon+lon*node_dist, wildfire_lat+lat*node_dist)) for lon, lat in burned])

# Style the layer
style = {'color': 'red', 'fillColor': '00000000'}

# Add the layer to the map
Map.addLayer(burned_feature_collection.style(**style), {}, 'Burned Area')

# To update the map in the notebook
Map  # This will display the updated map in the notebook cell

{(-11, 2), (-30, -2), (-19, -2), (-2, 14), (2, -25), (-2, 23), (2, -16), (2, -7), (-9, 2), (2, 2), (-17, -2), (2, 11), (-2, -11), (6, 2), (-13, -2), (-2, -2), (-1, -2), (-2, -1), (-1, -1), (-2, 7), (4, 2), (-2, 16), (2, -23), (-2, 25), (2, -14), (8, 2), (19, 2), (0, -2), (11, -2), (2, -5), (0, -1), (2, 4), (-2, -18), (15, -2), (26, -2), (-2, -9), (21, 2), (-2, 0), (-1, 0), (-2, 9), (2, 25), (-19, 2), (-30, 2), (2, -30), (-27, -2), (-2, 18), (17, -2), (28, -2), (2, -21), (2, -12), (-23, -2), (2, -3), (0, 0), (-2, -25), (-17, 2), (-2, -16), (-2, -7), (-2, 2), (-1, 2), (-13, 2), (-21, -2), (-10, -2), (-2, 11), (2, 27), (2, -28), (-6, -2), (5, -2), (2, -19), (2, -10), (0, 2), (11, 2), (-8, -2), (-2, -23), (-2, -14), (15, 2), (26, 2), (-4, -2), (7, -2), (18, -2), (-2, -5), (-2, 4), (2, 20), (22, -2), (2, 29), (2, -26), (-27, 2), (17, 2), (28, 2), (2, -17), (9, -2), (2, -8), (-23, 2), (-2, -30), (-20, -2), (13, -2), (24, -2), (-2, -21), (-2, -12), (-16, -2), (-2, -3), (2, 13), (2, 22), (-21,

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…